# Dataset preparation
Mnist dataset is divided into two parts:
*   train (60000 X 28 X28)
*   test  (10000 X 28 X28)

In this example we are using a Fully Connected Neural Network so the data sample is reshaped from 28X28 to 784. The data is normalized between 0 and 1. to_categorical function. Converts a class vector (integers) to binary class matrix.





In [ ]:
from keras.datasets import mnist
import numpy as np
from tensorflow.keras.utils import to_categorical
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(60000, 784)
X_train = X_train.astype('float32')
X_train /= 255
Y_train = to_categorical(y_train)

X_test = X_test.reshape(10000, 784)
X_test = X_test.astype('float32')
X_test /= 255
Y_test = to_categorical(y_test)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
from keras.datasets import mnist
import random
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import math
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.losses import categorical_crossentropy
from keras.callbacks import Callback
from tensorflow.keras.losses import MSE


# Preprocessing



*   X: X_train
*   y: Y_train
*   KBatch(X,y,K):  Divides the dataset into K baches.
*   RandomforestRank(X,y,k):  Ranks the features of each batch using random forest.
*   ConvertDistribute(RankList): Each vector converts the ranking into a statistical distribution.

*   JenssenShanon(DistributeVector): Calculates the Jensen-Shannon divergence of statistical distributions.

*  getInputWeight(X,y,k): The distribution with minimum divergence is chosen.



In [ ]:
def KBatch(X,y,K):
    index = []
    for i in range(0, len(y)):
        index.append(i)
    random.shuffle(index)
    Kx=[]
    Ky=[]
    step=round(len(y)/K)
    for i in range(0, K-1):
        inx=index[i*step:(i+1)*step]
        Kx.append(X[inx])
        Ky.append(y[inx])

    inx=index[(K-1)*step:len(y)]
    Kx.append(X[inx])
    Ky.append(y[inx])
    return(Kx,Ky)

def Rank(x):
    R=[];
    d=len(x)
    for i in range(0, d):
       r=1
       for j in range(0, d):
          if(x[i]<x[j]):
             r=r+1
       if findlist(R,r)>-1:
           r=r+1
       R.append(r)
    return(R)

def RandomforestRank(X,y,k):
    Kx,Ky = KBatch(X,y,k)
    print('Batching...',k)
    RankList=[]
    Weight=[]
    for i in range(0, k):
        rf = RandomForestRegressor()
        rf.fit(Kx[i], Ky[i])
        fi=rf.feature_importances_
        RankList.append(Rank(fi))
        Weight.append(fi)
        print('RandomForest...',i)
    return(RankList,Weight)


def ConvertDistribute(RankList):
  DistVector=[]
  d=len(RankList[0])
  ki=len(RankList)
  for k in range(0, ki):
    x=RankList[k]
    dv=[]
    for i in range(0, d):
      s=0
      for j in range(0, d-x[i]+1):
          s=s+1/(x[i]+j)
      s=(1+s)/(2*d)
      dv.append(s)
    DistVector.append(dv)
  return(DistVector)

def findlist(x,a):
    inx=-1
    for i in range(0, len(x)):
        if(x[i]==a):
           inx=i
    return(inx)


def JenssenShanon(DistributeVector):
  Dp=[]
  d=len(DistributeVector[0])
  k=len(DistributeVector)

  for i in range(0, d):
     p=0
     for j in range(0, k):
        p=p+DistributeVector[j][i]
     Dp.append(p/k)

  Djs=[]
  for i in range(0, k):
     s=0
     for j in range(0, d):
       p=DistributeVector[i][j]
       s=s+p*math.log(p/Dp[j])
     Djs.append(s/k)
  print('JenssenShanon...')
  return(Djs)


def getInputWeight(X,y,k):
  RankList,Weight = RandomforestRank(X,y,k)
  DistributeVector=ConvertDistribute(RankList)
  Djs=JenssenShanon(DistributeVector)
  i=np.argmin(Djs)
  return DistributeVector[i]


In [ ]:
Z0=getInputWeight(X_train,y_train,10)

Batching... 10
RandomForest... 0
RandomForest... 1
RandomForest... 2
RandomForest... 3
RandomForest... 4
RandomForest... 5
RandomForest... 6
RandomForest... 7
RandomForest... 8
RandomForest... 9
JenssenShanon...


# JSDropout Class
JSDropout is customized based on keras.dropout source. The input to the JSDropout is the dropout **rate** and the number of the network layer on which the dropout is performed.

The **mask** in the call() is a boolean vector with the length of the number of nodes in the dropout layer, which specifies active and inactive nodes with true and false.

**getJSDmask():** based on the JSDropout method , determines the list of active and inactive nodes.


**ZWeight** is the weight calculated by forward propagation.

In [ ]:

class JSDropout(Layer):
    def __init__(self,  rate,Zinx, **kwargs):
        super(JSDropout, self).__init__(**kwargs)
        self.rate = rate
        self.Zinx = Zinx


    def call(self, inputs, training=None):
        if training:
           mask=self.getJSDmask()
           return tf.where(mask, inputs, tf.zeros_like(inputs))
        return inputs

    def getJSDmask(self):
           global ZWeight
           Nweight=ZWeight[self.Zinx].copy()
           Nweight=[sorted(Nweight).index(x) for x in Nweight]
           alpha=1.0/(len(Nweight)-1)
           Nweight=list(map(lambda x : x*alpha, Nweight))
           j=0
           Rate=len(Nweight)*self.rate
           for i in range(len(Nweight)):
               if(j<Rate and Nweight[i]>random.random()):
                 Nweight[i]=True
                 j+=1
               else:
                 Nweight[i]=False
           Nweight=np.array([Nweight]).tolist()
           mask=tf.convert_to_tensor(Nweight)
           return(mask)


# Forward propagation

*   forwardPropogate(): Calculating the Zweight using Z0 and forward propagation.
*   JSCallback(Callback): Using the callback technique, forwardPropagation() is performed during training and updates the Zweights



In [ ]:
class JSCallback(Callback):
    #def on_train_batch_begin(self, batch, logs=None):
        #forwardPropogate()
        #print("Start batch {} forwardpropagation".format(batch))
    def on_epoch_begin(self, epoch, logs=None):
        forwardPropogate()
        print("forwardpropagation {}".format(epoch))



def forwardPropogate():
    global ZWeight
    global Z0
    Weights=[]
    Weights.append(Model.get_layer("L0").get_weights()[0])
    Weights.append(Model.get_layer("L1").get_weights()[0])
    ZWeight=[]
    Z=np.array(Z0)
    for i in range(int(len(Weights))):
       Z=np.matmul(Z, Weights[i])
       Z=activations.relu(Z).numpy()
       ZWeight.append(Z)


# Deep Neural Network Architecture

In [ ]:
Model = Sequential()
Model.add(Dense(200, activation='relu', input_shape=(784,),name="L0"))
Model.add(JSDropout(0.5,0))
Model.add(Dense(200, activation='relu',name="L1"))
Model.add(JSDropout(0.5,1))
Model.add(Dense(10, activation='softmax'))
Model.summary()
opt = Adam(learning_rate=0.001)
Model.compile(optimizer=opt, loss=MSE, metrics=['accuracy'])


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ L0 (Dense)                           │ (None, 200)                 │         157,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ js_dropout_18 (JSDropout)            │ (None, 200)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ L1 (Dense)                           │ (None, 200)                 │          40,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ js_dropout_19 (JSDropout)            │ (None, 200)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 10)                  │           2,010 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 199,210 (778.16 KB)

 Trainable params: 199,210 (778.16 KB)

 Non-trainable params: 0 (0.00 B)

# Train

In [ ]:
history=Model.fit(X_train, Y_train, batch_size=128, epochs=10, validation_split=0.2,callbacks=[JSCallback()])

forwardpropagation 0
Epoch 1/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7640 - loss: 0.0365 - val_accuracy: 0.9214 - val_loss: 0.0122
forwardpropagation 1
Epoch 2/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9279 - loss: 0.0112 - val_accuracy: 0.9408 - val_loss: 0.0091
forwardpropagation 2
Epoch 3/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9463 - loss: 0.0085 - val_accuracy: 0.9522 - val_loss: 0.0076
forwardpropagation 3
Epoch 4/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9581 - loss: 0.0066 - val_accuracy: 0.9587 - val_loss: 0.0065
forwardpropagation 4
Epoch 5/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9662 - loss: 0.0054 - val_accuracy: 0.9620 - val_loss: 0.0059
forwardpropagation 5
Epoch 6/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9715 - loss: 0.0047 - val_accuracy: 0.9638 - val_loss: 0.0057
forwardpropagation 6
Epoch 7/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9766 - loss: 0.0039

# Test

In [ ]:
train_results = Model.evaluate(X_train, Y_train, batch_size=128)
print("train loss, train acc:", train_results)
test_results = Model.evaluate(X_test, Y_test, batch_size=128)
print("test loss, test acc:", test_results)
gap=train_results[1]-test_results[1]
print("Gap:", gap)

469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9957 - loss: 7.5458e-04
train loss, train acc: [0.0014518426032736897, 0.991516649723053]
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9688 - loss: 0.0051
test loss, test acc: [0.004356123507022858, 0.9735000133514404]
Gap: 0.01801663637161255


In [ ]:
print(Z0)

[0.000816770183786111, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.0008409013664243148, 0.0008397844573139107, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.0008375564901925255, 0.000815694711438656, 0.0008420202350243866, 0.0008276258492575915, 0.0008465154426182652, 0.0008753519516393771, 0.

In [ ]:
Z0=[0.000816770183786111, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.0008409013664243148, 0.0008397844573139107, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.0008375564901925255, 0.000815694711438656, 0.0008420202350243866, 0.0008276258492575915, 0.0008465154426182652, 0.0008753519516393771, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.0008386695008418113, 0.0008510425596557019, 0.0009530550044654991, 0.0008578940661789977, 0.0009307902528046561, 0.0009543892201601451, 0.001033945553511604, 0.0010958658812616584, 0.0010131069029922436, 0.0010400266942230472, 0.0009858727835146122, 0.0010043803610146774, 0.0009180474734408427, 0.0009030847953767483, 0.0008800892430129921, 0.0008969524894226746, 0.000815694711438656, 0.0008287216484019915, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.0008729964367888194, 0.0008567470246285646, 0.00096245366733281, 0.001014573006675096, 0.0010875832357101242, 0.0010794067807328722, 0.0010908833970004908, 0.001111051235090445, 0.0010826647942310842, 0.001112761034291627, 0.0010160424884770333, 0.0011008875980413138, 0.0011076453272727725, 0.0010570551394978567, 0.0010649482449070608, 0.0009733673857906591, 0.0009957716161312531, 0.000895733072401564, 0.0009346632700776795, 0.0009105222546091136, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.0008777161987193676, 0.0008476442127103728, 0.0008613476211646105, 0.0009557262329526835, 0.0009761252489145909, 0.001041556082956958, 0.0010249312012170313, 0.0011987779240822507, 0.0014442750007218703, 0.0022614786028074757, 0.001786583720792822, 0.0014558972631148856, 0.0012392196523094052, 0.0014024857697683354, 0.0011909884580084444, 0.0010279218505888955, 0.0012007402474731455, 0.0010942007243372699, 0.0010586259648723414, 0.0009678871814423753, 0.0009117703076463756, 0.0008601943569656216, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.0008232613843440886, 0.000815694711438656, 0.0009055543237395757, 0.0008625029746103366, 0.0010014978483985871, 0.0010992092951412063, 0.0012735273250691873, 0.0015963778889594772, 0.001665809265745418, 0.002439780011983868, 0.0031403273670585576, 0.0023999087796220562, 0.002993955770174189, 0.0022719336044802756, 0.0016536996682634385, 0.0015174151704936325, 0.0013944467452955168, 0.0011756892626054597, 0.0011144754297272204, 0.0011552347363551273, 0.0011318777292330109, 0.0009986283055009545, 0.0008636604248862727, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.0008210903015731155, 0.0009092766391754401, 0.0009929276656533525, 0.0010324342854972895, 0.0010975353972355873, 0.0011700453794041715, 0.0013607364398156637, 0.0013734919500419504, 0.0014797971627418544, 0.0020741340719819716, 0.0018706891381206194, 0.0018650947951202612, 0.0020151095037808024, 0.001816838489796692, 0.0015927335740906722, 0.0012913990219916589, 0.0012268156072946746, 0.0012561428215012633, 0.0014889956953804408, 0.001376073954503654, 0.00125400270370918, 0.00111619444617477, 0.0009477458801789877, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.0009043183642394966, 0.0008933012108892384, 0.000933369649181045, 0.0009490690235442175, 0.0010539250483950766, 0.0011283590736999335, 0.0013168201352945874, 0.0015819239355549378, 0.0013335412447640295, 0.0016149185349438608, 0.002089785093004653, 0.0019741703808809764, 0.0021140062286351467, 0.0036940448372301126, 0.0023269452122772457, 0.0022410691333361234, 0.0017719780747122579, 0.0013215530280386327, 0.001358215668661352, 0.0012476251054471283, 0.00123092354130948, 0.0014677352634640335, 0.001140760250111946, 0.0009638077121354446, 0.0008706495899008931, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.0008364454185861825, 0.0008765329796061195, 0.0009385599515378415, 0.0010029374761684313, 0.0010385009643138587, 0.0011213794723911737, 0.0012779485175657937, 0.001275734090128152, 0.0012434085312690996, 0.0013432967592618779, 0.002097757031780163, 0.0022933722646901224, 0.001994311503539447, 0.003228398309721335, 0.002174378537716033, 0.0030629526671786056, 0.0017122886384625384, 0.0017530100695186844, 0.001389143145639887, 0.0012891374790766203, 0.001245513333586066, 0.0011929269233641913, 0.0012582901450771584, 0.0009775086656218378, 0.0008896707177226687, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.0009231140732157251, 0.0009451078004103523, 0.000916787087863687, 0.0010508102447182239, 0.0011354159058148165, 0.0011890558667901386, 0.0013838832873858007, 0.0014588362267187601, 0.0014161135133137317, 0.004618789735189295, 0.0019112869778074822, 0.003277456394493705, 0.0021927321524538535, 0.002688148666671104, 0.0022511922301939142, 0.0027299799152995874, 0.0023871536775812397, 0.001811695303489911, 0.0016457524870617654, 0.0013121221075595077, 0.001216660065268867, 0.0012207028969886556, 0.0011179181086127179, 0.0010354604112361666, 0.0008872618200051482, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.0009067926831610141, 0.001036978875764835, 0.0011336443638647032, 0.0010777839942136844, 0.0012027086274177161, 0.001504627699156416, 0.0016224883950941764, 0.0026301354045480726, 0.0042999121841688845, 0.0031828443738612785, 0.0019173035353739056, 0.0020819115732263714, 0.002668218819732327, 0.002774748438472404, 0.002560178757322359, 0.0021058298811730855, 0.0023746486755804393, 0.0022116296368207343, 0.001603729593843145, 0.001365808071010353, 0.0011775817109794678, 0.0013005262535767848, 0.000946425476241015, 0.0009243870374912557, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.0008178474728098286, 0.0009282212467671141, 0.0011794797916403039, 0.0011371923824221725, 0.0014828486225602316, 0.003027521828176338, 0.002962068015072149, 0.001980813663193902, 0.002527882184847215, 0.0029027098562241947, 0.0021306802769284214, 0.0017300668633877404, 0.002594198410702915, 0.002139183678288966, 0.0018877727720234918, 0.003602936965509995, 0.0023155567283122316, 0.002221292592912262, 0.0017255756302747766, 0.001450059657241821, 0.0012604447231245933, 0.0010633617894293475, 0.0009816770128122001, 0.0009359595202850795, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.0008884651315184326, 0.0009398641542005019, 0.0011248597249916108, 0.0012413106526439652, 0.0016074374723433823, 0.001571294465138624, 0.0020081012059561783, 0.0017624235571359534, 0.0020513549517280185, 0.0023385407595870786, 0.002001169085281822, 0.0016907408903909028, 0.0018325216947853433, 0.002611913830204049, 0.002282562856180956, 0.0031004676731810066, 0.0024536442533325815, 0.0019482683591436187, 0.0015505526844933229, 0.001350712590535417, 0.0013482310531733905, 0.001282400573722678, 0.000988685378342782, 0.000900624792695559, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.0008556020423986888, 0.000815694711438656, 0.000815694711438656, 0.0010446259073881208, 0.0010554881736451766, 0.0013287188827145785, 0.0012982321704759186, 0.0015337687255969669, 0.0016418157271726249, 0.0017768095527580215, 0.0018935705456784083, 0.0033306026529971076, 0.0019546459101640267, 0.0025769617863234335, 0.002848408244275067, 0.0023503510392545008, 0.0017211158043863796, 0.0013074584349195208, 0.0013359661691063902, 0.0014471607704143628, 0.0012801706607784794, 0.0015370903667534293, 0.0011231172247128107, 0.0009747448266158012, 0.0008331237707552346, 0.0008298193336034386, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000854459112108293, 0.000815694711438656, 0.000991510432093262, 0.0010072759611223765, 0.0010892311817102297, 0.0012497438931615497, 0.0014529717809954321, 0.0018540512097700794, 0.0019053266497510261, 0.0038003373542369154, 0.002931698724498777, 0.001899421509917315, 0.001859549098580776, 0.002543826062398236, 0.002708721411898227, 0.005256544837230112, 0.0012247715204291593, 0.0012288662667867675, 0.001657710706641054, 0.0012626066048264267, 0.001507800610111843, 0.0011059491700864937, 0.0010175153640013769, 0.0009080334518420274, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000878901617124648, 0.0009503949177272546, 0.0009830725381995982, 0.0010729402638815016, 0.0011534073578392508, 0.0017576994452689846, 0.0015855068293866275, 0.0019419539521927192, 0.002029361459244636, 0.0035232175777548946, 0.002022195671581256, 0.0020439391947275442, 0.001967595586014576, 0.0017166869495110963, 0.0013786664549184544, 0.0014356952206507438, 0.0015539631395844501, 0.00137092035688856, 0.0020664502755718414, 0.00118138353821356, 0.0011625970652670127, 0.0010810337070135884, 0.0009651646378844677, 0.0008453886668196066, 0.0008431410700016111, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000892088748718058, 0.0009788950897567093, 0.0010101847681063367, 0.001470729418872676, 0.0024129241898677876, 0.0017079204528321222, 0.0015206525060369866, 0.0016865173466687777, 0.001694992591071175, 0.0023623841543873466, 0.001961087880891711, 0.0016781532554182423, 0.0016111670343436203, 0.004087327150155279, 0.0018065932626735845, 0.0015748179739896785, 0.002202110903954453, 0.003927888374645076, 0.0011871291142159974, 0.0011948712986752914, 0.0011425516970277909, 0.0010058265177086475, 0.000971992913587985, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000901853607921071, 0.0010681330443879505, 0.0014188743579113107, 0.001801531714244689, 0.0024823110230707793, 0.001600043148166608, 0.001492091593934037, 0.0018763329885811576, 0.002036608676313282, 0.0023043680423115156, 0.0016617471313375147, 0.001987526874794332, 0.0022311042098667358, 0.0033885803895462737, 0.0017915275587931385, 0.0014272293344587037, 0.002822898040193435, 0.001767182923569094, 0.0013408438550232594, 0.0012186782776170974, 0.0011479564300350068, 0.0010234411192029172, 0.0009424805881406918, 0.0008945159825121732, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.0008353362792782856, 0.0008981742424917184, 0.0009802845344234646, 0.001626307287920768, 0.001882027230563665, 0.00215653836173954, 0.0015471603701207655, 0.0015573919304556374, 0.002147801990478707, 0.0021834893248880442, 0.001734589949217817, 0.0016497136988756838, 0.002058857952928499, 0.0015608392553315335, 0.0014952025944317974, 0.0013384003488851716, 0.0015014704956809665, 0.001543786004501502, 0.0014051881218956268, 0.0015304642950164445, 0.0013812695369676007, 0.0012106434680374976, 0.0010492585291414823, 0.000966524456865365, 0.0008836654481764651, 0.0009372584105132685, 0.000815694711438656, 0.0008671454331523456, 0.0008254398832924559, 0.0008487749841678922, 0.0011968216200882603, 0.0014328607535305624, 0.0034523558997503583, 0.0024262107544936375, 0.0015141941851297901, 0.001567790316226312, 0.0019233773934885807, 0.0018220234906262919, 0.0015271768975832442, 0.001540429398701287, 0.0014617887966356159, 0.0015239063585984194, 0.0012350581071267285, 0.0015643053156687123, 0.0014133645689083834, 0.0013632672140301107, 0.0014244321629585247, 0.001827250991462692, 0.0012371354853092718, 0.0011266069992437776, 0.0009943480556356262, 0.0009844711239496875, 0.0008993983405762496, 0.0009193103548310225, 0.000815694711438656, 0.000815694711438656, 0.0008499077641004691, 0.0008718219338384681, 0.0010523657449671038, 0.0012959463098951274, 0.001929509654085127, 0.0021222887624278847, 0.0015891099655563495, 0.0016698974394764486, 0.0018431956072074548, 0.0016379031191846442, 0.0016186922337725047, 0.0016740119885218733, 0.0013917894323703468, 0.001314466795434658, 0.0013683590914185166, 0.0014859147528585046, 0.0014079019733936728, 0.0014983288449319971, 0.001476760233684517, 0.0017035819827502117, 0.0010713338278562099, 0.0010602006688279978, 0.0009900963409579164, 0.0008860607746152407, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.0008265319297000599, 0.0008694793970531117, 0.0008848619868294498, 0.0009570660545956264, 0.0011738024131911376, 0.0014737376976558876, 0.001682321589418509, 0.002874981373526768, 0.002648892907549273, 0.0017483549227884595, 0.0015109893856220472, 0.001397115176684809, 0.001309786008284633, 0.0014414022299919567, 0.0013311255057411476, 0.0013191821912280717, 0.0011570673659586927, 0.0013865077939785614, 0.001848600311462038, 0.0013557048218029238, 0.0012227339641926072, 0.0010294224508289916, 0.0010264247728845742, 0.000929504456227961, 0.0008442638782798521, 0.0008200075255085301, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.0009130208078464556, 0.0008243497036308476, 0.0010430891481060947, 0.0010859395369935243, 0.0011461497583578374, 0.001264775839867382, 0.0016992728266553413, 0.0027983689978072494, 0.001781677912315585, 0.0012868839274793025, 0.0014106274225476933, 0.0013051393254575542, 0.001323932711255203, 0.0013263213071429962, 0.0012518697435016854, 0.0012713281695449087, 0.0011497682343021682, 0.0011719211297042916, 0.0011196464422225845, 0.0010617792705657225, 0.00097062139723951, 0.0009320786469501931, 0.0009155291882541389, 0.0008309189113655779, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.0008189265846576134, 0.0009437928414370722, 0.001046166378649089, 0.0009597541718145327, 0.0010761653264420071, 0.0012046831014178425, 0.0012691365712561085, 0.0013532038214027636, 0.0014216472061810538, 0.0015783610578899052, 0.001743733509005555, 0.0019357014511923194, 0.0017391453428038228, 0.0014300388283002932, 0.0013028286185661018, 0.001214648219521104, 0.0012066637073248013, 0.0012126427003322964, 0.0010665386566328485, 0.0010189916489598049, 0.001011644161932517, 0.0010219545105734516, 0.000864819979617256, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.0008683113474156743, 0.0009205757419382464, 0.0010204713591733566, 0.0010843000637234966, 0.0011607484997538508, 0.0011515852004048484, 0.0011663103488379501, 0.0013457591341732322, 0.0011832929846268558, 0.0014385423416419978, 0.0014647550994358056, 0.0013997948199706946, 0.001232987473678544, 0.0011681751298380696, 0.0012669524784409004, 0.0012936686131021242, 0.0011852081651134649, 0.0010745507565634229, 0.0010925399037590385, 0.001008728706229531, 0.0008590431744709632, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.0008221749190935933, 0.000832020388225752, 0.0009142737648249248, 0.0009584086969157124, 0.001104257511990895, 0.001144348190272976, 0.00120865048334362, 0.001634014368562444, 0.0018378363206356832, 0.0025123271823584144, 0.0027519714705423744, 0.002497142537071728, 0.0017965100205278323, 0.002467816588933488, 0.002165396071490107, 0.0016301491861258333, 0.001284638310922821, 0.0011644510045171314, 0.0011093460075448814, 0.000969252824059165, 0.0008824711502325685, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.0008342290655594647, 0.0009269406140320924, 0.0008908785872341097, 0.0009611024912691645, 0.0009517235741898397, 0.0010000614630336305, 0.0009971983613259751, 0.0010697314281023385, 0.0011389738212546888, 0.0011589052769155827, 0.0010309265902205972, 0.001102570329181263, 0.001130115974807484, 0.0010477105798646841, 0.0009872775304354071, 0.000921843644725604, 0.000815694711438656, 0.0009411710294096019, 0.000815694711438656, 0.0008521793780194109, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.0008741731067187841, 0.0008533182264159124, 0.000815694711438656, 0.0008812790846212772, 0.0008659816464697893, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.0009256625476953372, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656, 0.000815694711438656]
